<a href="https://www.kaggle.com/code/singhayush16/spaceship-titanic?scriptVersionId=140310470" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


# **Training Part**

In [25]:
train=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [27]:
train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [30]:
train.dropna(inplace=True)

In [31]:
df=train.drop(['PassengerId','Name'],axis=1)
df

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False
8689,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False
8690,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True
8691,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False


In [32]:
df.HomePlanet.value_counts()

Earth     3566
Europa    1673
Mars      1367
Name: HomePlanet, dtype: int64

In [33]:
df.HomePlanet.replace({'Earth':0,'Europa':1,'Mars':2},inplace=True)

In [34]:
df

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,0,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,1,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,1,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,0,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,1,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False
8689,0,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False
8690,0,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True
8691,1,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False


In [44]:
df[['deck', 'num', 'side']]=df.Cabin.str.split('/',expand=True)

In [46]:
df.drop(['Cabin'],axis=1,inplace=True)

In [49]:
df.drop(['new'],axis=1,inplace=True)

In [50]:
df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,deck,num,side
0,1,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,0,P
1,0,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,0,S
2,1,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,0,S
3,1,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,0,S
4,0,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,1,S


In [53]:
df.side.value_counts()

S    3345
P    3261
Name: side, dtype: int64

In [54]:
df.replace({'side':{'S':0,'P':1}},inplace=True)

In [51]:
df.deck.value_counts()

F    2152
G    1973
E     683
B     628
C     587
D     374
A     207
T       2
Name: deck, dtype: int64

In [55]:
df.replace({'deck':{'F':0,'G':1,'E':2,'B':2,'C':2,'D':2,'A':2,'T':2}},inplace=True)

In [11]:
df.VIP.value_counts()

False    8291
True      199
Name: VIP, dtype: int64

In [64]:
df.replace({'VIP':{False:0,True:1}},inplace=True)

In [10]:
df.Destination.value_counts()

TRAPPIST-1e      5915
55 Cancri e      1800
PSO J318.5-22     796
Name: Destination, dtype: int64

In [57]:
df.replace({'Destination':{'TRAPPIST-1e':0,'55 Cancri e':1,'PSO J318.5-22':1}},inplace=True)

In [9]:
df.CryoSleep.value_counts()

False    5439
True     3037
Name: CryoSleep, dtype: int64

In [62]:
df.replace({'CryoSleep':{False:0,True:1}},inplace=True)

In [65]:
df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,deck,num,side
0,1,0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,False,2,0,1
1,0,0,0,24.0,0,109.0,9.0,25.0,549.0,44.0,True,0,0,0
2,1,0,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,2,0,0
3,1,0,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,2,0,0
4,0,0,0,16.0,0,303.0,70.0,151.0,565.0,2.0,True,0,1,0


In [66]:
y=df['Transported']
X=df.drop(['Transported'],axis=1)

In [67]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=2529)

In [68]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((4954, 13), (1652, 13), (4954,), (1652,))

In [87]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(max_iter=472)
model.fit(X_train,y_train)

LogisticRegression(max_iter=472)

In [88]:
y_pred=model.predict(X_test)

In [89]:
model.score(X_test,y_test)

0.8044794188861986

In [74]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [90]:
accuracy_score(y_test,y_pred)

0.8044794188861986

In [91]:
confusion_matrix(y_test,y_pred)

array([[645, 184],
       [139, 684]])

# **Testing Part**

In [92]:
test=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [93]:
test.isnull().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [106]:
df_test=test

In [108]:
df_test.HomePlanet.replace({'Earth':0,'Europa':1,'Mars':2},inplace=True)

In [110]:
df_test[['deck', 'num', 'side']]=df_test.Cabin.str.split('/',expand=True)

In [112]:
df_test.drop(['Cabin'],axis=1,inplace=True)

In [114]:
df_test.replace({'deck':{'F':0,'G':1,'E':2,'B':2,'C':2,'D':2,'A':2,'T':2}},inplace=True)

In [116]:
df_test.replace({'VIP':{False:0,True:1}},inplace=True)

In [118]:
df_test.replace({'Destination':{'TRAPPIST-1e':0,'55 Cancri e':1,'PSO J318.5-22':1}},inplace=True)

In [122]:
df_test.replace({'side':{'S':0,'P':1}},inplace=True)

In [124]:
df_test.replace({'CryoSleep':{False:0,True:1}},inplace=True)

In [125]:
df_test.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,deck,num,side
0,0013_01,0.0,1.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,1.0,3,0.0
1,0018_01,0.0,0.0,0.0,19.0,0.0,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,0.0,4,0.0
2,0019_01,1.0,1.0,1.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,2.0,0,0.0
3,0021_01,1.0,0.0,0.0,38.0,0.0,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,2.0,1,0.0
4,0023_01,0.0,0.0,0.0,20.0,0.0,10.0,0.0,635.0,0.0,0.0,Brence Harperez,0.0,5,0.0


In [133]:
df_test.isnull().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
deck            100
num             100
side            100
dtype: int64

In [135]:
df_test.dropna(inplace=True)

In [136]:
X_Test=df_test.drop(['PassengerId','Name'],axis=1)

In [138]:
df_test['Transported']=model.predict(X_Test)

In [140]:
df_test[['PassengerId','Transported']].to_csv('submission.csv',index_label='PassengerId')